In [1]:
!pip install -q torch==2.0.1 torchtext==0.15.2


In [2]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 59.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [4]:
!kaggle datasets download -d devicharith/language-translation-englishfrench

Dataset URL: https://www.kaggle.com/datasets/devicharith/language-translation-englishfrench
License(s): CC0-1.0
language-translation-englishfrench.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/language-translation-englishfrench.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
import pandas as pd
import torch
import torch.nn as nn
import spacy
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import GloVe
from torchtext.datasets import AG_NEWS
from torch.utils.data import DataLoader
from torchtext.transforms import VocabTransform, PadTransform
from torchtext.vocab import build_vocab_from_iterator
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_csv('/content/eng_-french.csv')
df.head()

,English words/sentences,French words/sentences
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !


In [8]:
def get_rows(i,j):
  return df.iloc[i:j, :]

df = get_rows(120000,150000)

In [9]:
df.shape

(30000, 2)

In [11]:
df.rename(columns={'English words/sentences':'English','French words/sentences':'French'},inplace=True)

In [12]:
max_length = df['French'].apply(lambda x: len(x.split())).max()

print(f"The maximum length of a sentence in the 'english' column is: {max_length}")

The maximum length of a sentence in the 'english' column is: 18


In [13]:
english_texts = df["English"].tolist()
french_summaries = df["French"].tolist()
train_en, test_en, train_fr, test_fr = train_test_split(english_texts, french_summaries, test_size=0.2, random_state=42)


In [14]:
import os
import urllib.request

def download_fasttext_vectors(url, dest_path):

    os.makedirs(os.path.dirname(dest_path), exist_ok=True)

    if not os.path.exists(dest_path):
        print(f"Downloading {url}...")
        urllib.request.urlretrieve(url, dest_path)
        print("Download completed.")
    else:
        print(f"{dest_path} already exists.")


english_vectors_url = "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz"
french_vectors_url = "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz"


english_vectors_path = ".vector_cache/cc.en.300.vec.gz"
french_vectors_path = ".vector_cache/cc.fr.300.vec.gz"


download_fasttext_vectors(english_vectors_url, english_vectors_path)
download_fasttext_vectors(french_vectors_url, french_vectors_path)

.vector_cache/cc.en.300.vec.gz already exists.
.vector_cache/cc.fr.300.vec.gz already exists.


In [15]:
import gzip
import shutil

def extract_gz(source_path, dest_path):
    with gzip.open(source_path, 'rb') as f_in:
        with open(dest_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)


extract_gz(".vector_cache/cc.en.300.vec.gz", ".vector_cache/cc.en.300.vec")
extract_gz(".vector_cache/cc.fr.300.vec.gz", ".vector_cache/cc.fr.300.vec")


In [16]:
from torchtext.vocab import Vectors

english_vectors_path = ".vector_cache/cc.en.300.vec"
french_vectors_path = ".vector_cache/cc.fr.300.vec"


english_vectors = Vectors(name=english_vectors_path)
french_vectors = Vectors(name=french_vectors_path)


In [17]:
spacy_en = spacy.load("en_core_web_sm")
spacy_fr = spacy.load("fr_core_news_sm")

In [18]:
def tokenize_en(text):
    return [tok.text.lower() for tok in spacy_en.tokenizer(text)]

def tokenize_fr(text):
    return [tok.text.lower() for tok in spacy_fr.tokenizer(text)]

tokenizer_en = get_tokenizer(tokenize_en)
tokenizer_fr = get_tokenizer(tokenize_fr)
def yield_tokens(data_iter, tokenizer):
    for text in data_iter:
        yield tokenizer(text)
all_data = english_texts + french_summaries
vocab_en = build_vocab_from_iterator(yield_tokens(english_texts, tokenize_en), specials=["<unk>", "<pad>"])
vocab_fr = build_vocab_from_iterator(yield_tokens(french_summaries, tokenize_fr), specials=["<unk>", "<pad>"])
vocab_en.set_default_index(vocab_en["<unk>"])
vocab_fr.set_default_index(vocab_fr["<unk>"])

In [19]:
class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, english_texts, french_texts):
        self.english_texts = english_texts
        self.french_texts = french_texts

    def __getitem__(self, idx):
        english_text = self.english_texts[idx]
        french_text = self.french_texts[idx]
        english_tensor = torch.tensor([vocab_en[token] for token in tokenize_en(english_text)], dtype=torch.long)
        french_tensor = torch.tensor([vocab_fr[token] for token in tokenize_fr(french_text)], dtype=torch.long)
        return english_tensor, french_tensor

    def __len__(self):
        return len(self.english_texts)

In [22]:
import torch.nn.functional as F

def collate_fn(batch):
    english_batch, french_batch = zip(*batch)

    max_length = max(max(len(seq) for seq in english_batch), max(len(seq) for seq in french_batch))


    english_batch = [F.pad(seq, (0, max_length - len(seq))) for seq in english_batch]
    french_batch = [F.pad(seq, (0, max_length - len(seq))) for seq in french_batch]

    return torch.stack(english_batch), torch.stack(french_batch)

In [23]:
train_dataset = TranslationDataset(train_en, train_fr)
test_dataset = TranslationDataset(test_en, test_fr)


train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)

In [24]:
def create_embedding_matrix(vocab, vectors, embedding_dim):
    embedding_matrix = torch.zeros((len(vocab), embedding_dim))

    for idx, token in enumerate(vocab.get_itos()):
        if token in vectors.stoi:
            embedding_matrix[idx] = vectors[token]
        else:
            embedding_matrix[idx] = torch.randn(embedding_dim)

    return embedding_matrix


embedding_dim = english_vectors.dim
english_embedding_matrix = create_embedding_matrix(vocab_en, english_vectors, embedding_dim)
french_embedding_matrix = create_embedding_matrix(vocab_fr, french_vectors, embedding_dim)


class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, pretrained_embeddings):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers


        self.embedding = nn.Embedding(input_size, embedding_size)
        self.embedding.weight.data.copy_(pretrained_embeddings)


        self.embedding.weight.requires_grad = False

        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)

    def forward(self, x):
        embedding = self.dropout(self.embedding(x))
        outputs, (hidden, cell) = self.rnn(embedding)
        return hidden, cell


class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, p, pretrained_embeddings):
        super(Decoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers


        self.embedding = nn.Embedding(input_size, embedding_size)
        self.embedding.weight.data.copy_(pretrained_embeddings)


        self.embedding.weight.requires_grad = False

        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        x = x.unsqueeze(0)
        embedding = self.dropout(self.embedding(x))
        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
        predictions = self.fc(outputs)
        predictions = predictions.squeeze(0)
        return predictions, hidden, cell


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(vocab_fr)

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        hidden, cell = self.encoder(source)
        x = target[0]

        for t in range(1, target_len):
            output, hidden, cell = self.decoder(x, hidden, cell)
            outputs[t] = output
            best_guess = output.argmax(1)
            x = target[t] if random.random() < teacher_force_ratio else best_guess

        return outputs

In [25]:
input_size_encoder = len(vocab_en)
input_size_decoder = len(vocab_fr)
output_size = len(vocab_fr)
embedding_size = embedding_dim
hidden_size = 512
num_layers = 2
dropout = 0.5


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


english_embedding_matrix = english_embedding_matrix.to(device)
french_embedding_matrix = french_embedding_matrix.to(device)


encoder = Encoder(input_size_encoder, embedding_size, hidden_size, num_layers, dropout, english_embedding_matrix).to(device)
decoder = Decoder(input_size_decoder, embedding_size, hidden_size, output_size, num_layers, dropout, french_embedding_matrix).to(device)
model = Seq2Seq(encoder, decoder).to(device)


criterion = nn.CrossEntropyLoss(ignore_index=vocab_fr['<pad>']).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 120
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for english_batch, french_batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):

        english_batch = english_batch.to(device)
        french_batch = french_batch.to(device)


        optimizer.zero_grad()
        output = model(english_batch, french_batch)


        output = output[1:].reshape(-1, output.shape[2])
        target = french_batch[1:].reshape(-1)


        loss = criterion(output, target)
        loss.backward()


        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()


        epoch_loss += loss.item()


    avg_epoch_loss = epoch_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_epoch_loss:.4f}")


Using device: cuda


Epoch 1/120: 100%|██████████| 3000/3000 [01:07<00:00, 44.51it/s]


Epoch [1/120], Loss: 4.7711


Epoch 2/120: 100%|██████████| 3000/3000 [01:07<00:00, 44.75it/s]


Epoch [2/120], Loss: 4.5715


...

Epoch [118/120], Loss: 3.3678


Epoch 119/120: 100%|██████████| 3000/3000 [01:13<00:00, 40.87it/s]


Epoch [119/120], Loss: 3.3685


Epoch 120/120: 100%|██████████| 3000/3000 [01:12<00:00, 41.48it/s]

Epoch [120/120], Loss: 3.3718


In [26]:
def evaluate_model(model, data_loader, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for english_batch, french_batch in data_loader:
            english_batch = english_batch.to(device)
            french_batch = french_batch.to(device)


            output = model(english_batch, french_batch)


            output = output[1:].reshape(-1, output.shape[2])
            target = french_batch[1:].reshape(-1)


            loss = criterion(output, target)
            epoch_loss += loss.item()

    avg_loss = epoch_loss / len(data_loader)
    return avg_loss


test_loss = evaluate_model(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}")

Test Loss: 4.2269


In [27]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data_loader, model, vocab_fr, max_len=50):
    model.eval()
    targets = []
    outputs = []

    with torch.no_grad():
        for english_batch, french_batch in data_loader:
            english_batch = english_batch.to(device)
            french_batch = french_batch.to(device)


            output = model(english_batch, french_batch, 0)


            output = output.argmax(2)
            for i in range(output.shape[0]):
                predicted_tokens = [vocab_fr.get_itos()[token] for token in output[i].cpu().numpy()]
                target_tokens = [vocab_fr.get_itos()[token] for token in french_batch[i].cpu().numpy()]


                predicted_tokens = [token for token in predicted_tokens if token not in ['<sos>', '<eos>', '<pad>']]
                target_tokens = [token for token in target_tokens if token not in ['<sos>', '<eos>', '<pad>']]

                outputs.append(predicted_tokens)
                targets.append([target_tokens])


    bleu = bleu_score(outputs, targets)
    return bleu


bleu = calculate_bleu(test_loader, model, vocab_fr)
print(f"BLEU Score: {bleu:.4f}")


BLEU Score: 0.2574
